In [3]:
import sys

from delta.tables import DeltaTable
from pyspark.sql import SparkSession
from minio import Minio
from utils.common import get_spark_session, get_minio_client
from utils.bronze import Bronze
from utils.silver import Silver
from utils.gold import Gold

username = 'camonco'

spark = get_spark_session(
        f"{username}_ETL",
        username,
    )

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f8d62f2f-bd15-4060-87e2-959a4c81de47;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 1998ms :: artifacts dl 10ms
	:: modules in use:
	io.delta#delta-core_2.12;2.4.0 from central in [default]
	io.delta#delta-storage;2.4.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.4 from central in [default]
	org.wildfly.openssl#wildfly-openssl;1.0.7.Final from central in [default]
	-------------------------

In [6]:
source = 'shopee'

spark.sql("DROP TABLE IF EXISTS gold_camonco.shopee_fact_sales")


DataFrame[]

In [8]:

query = f"""
                SELECT
                    monotonically_increasing_id() AS sale_key,
                    date_dim.date_key AS date_key,
                    product_dim.product_key AS product_key,
                    customer_dim.customer_key AS customer_key,
                    promotion_dim.promotion_key AS promotion_key,
                    shipment_dim.shipment_key AS shipment_key,
                    silver.order_number,
                    silver.order_quantity,
                    silver.unit_price,
                    silver.unit_discount,
                    silver.sales_amount,
                    CAST(silver.order_date AS DATE) AS order_date,
                    silver.ship_date,
                    silver.payment_date
                FROM
                    silver_{username}.{source}_sales_silver AS silver
                LEFT JOIN
                    gold_{username}.dim_date AS date_dim
                ON
                    CAST(date_format(silver.order_date, 'yyyyMMdd') AS INT) = date_dim.date_key
                LEFT JOIN
                    gold_{username}.dim_product AS product_dim
                ON
                    silver.product_key = product_dim.product_key
                LEFT JOIN
                    gold_{username}.dim_customer AS customer_dim
                ON
                    silver.customer_name = customer_dim.customer_name
                LEFT JOIN
                    gold_{username}.dim_promotion AS promotion_dim
                ON
                    silver.promotion = promotion_dim.promotion
                LEFT JOIN
                    gold_{username}.dim_shipment AS shipment_dim
                ON silver.shipping_company = shipment_dim.shipping_company
            """

fact_sales_df = spark.sql(query)

            # Save fact_sales DataFrame to Delta Lake with partitioning and Z-Ordering
fact_sales_df.write \
                .format("delta") \
                .mode("overwrite") \
                .partitionBy("date_key") \
                .save(f"s3a://{username}/gold/shopee_fact_sales")

            # Create Delta table for easy querying
spark.sql(f"""
                CREATE TABLE IF NOT EXISTS gold_{username}.shopee_fact_sales
                USING DELTA 
                LOCATION 's3a://{username}/gold/shopee_fact_sales'
            """)

            # Optimize and Z-Order the table
spark.sql(f"""
                OPTIMIZE gold_{username}.shopee_fact_sales
                ZORDER BY (product_key, customer_key)
            """)

24/06/03 05:44:05 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
24/06/03 05:47:00 WARN HiveExternalCatalog: Couldn't find corresponding Hive SerDe for data source provider delta. Persisting data source table `spark_catalog`.`gold_camonco`.`shopee_fact_sales` into Hive metastore in Spark SQL specific format, which is NOT compatible with Hive.
24/06/03 05:47:01 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.
24/06/03 05:53:09 WARN S3AInstrumentation: Closing output stream statistics while data is still marked as pending upload in OutputStreamStatistics{counters=((multipart_upload_completed.failures=0) (action_executor_acquired.failures=0) (action_executor_acquired=0) (stream_write_total_time=0) (op_hflush=0) (object_multipart_aborted.failures=0) (op_abort=0) (stream_write_total_

DataFrame[path: string, metrics: struct<numFilesAdded:bigint,numFilesRemoved:bigint,filesAdded:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,filesRemoved:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,partitionsOptimized:bigint,zOrderStats:struct<strategyName:string,inputCubeFiles:struct<num:bigint,size:bigint>,inputOtherFiles:struct<num:bigint,size:bigint>,inputNumCubes:bigint,mergedFiles:struct<num:bigint,size:bigint>,numOutputCubes:bigint,mergedNumCubes:bigint>,numBatches:bigint,totalConsideredFiles:bigint,totalFilesSkipped:bigint,preserveInsertionOrder:boolean,numFilesSkippedToReduceWriteAmplification:bigint,numBytesSkippedToReduceWriteAmplification:bigint,startTimeMs:bigint,endTimeMs:bigint,totalClusterParallelism:bigint,totalScheduledTasks:bigint,autoCompactParallelismStats:struct<maxClusterActiveParallelism:bigint,minClusterActiveParallelism:bigint,maxSessionActiveParallelism:bigint,minSessionActiveParallelism:bigint>,de

In [9]:
df = spark.sql("select * from gold_camonco.shopee_fact_sales limit 10")
df.printSchema()

root
 |-- sale_key: long (nullable = true)
 |-- date_key: integer (nullable = true)
 |-- product_key: string (nullable = true)
 |-- customer_key: long (nullable = true)
 |-- promotion_key: long (nullable = true)
 |-- shipment_key: long (nullable = true)
 |-- order_number: string (nullable = true)
 |-- order_quantity: integer (nullable = true)
 |-- unit_price: float (nullable = true)
 |-- unit_discount: float (nullable = true)
 |-- sales_amount: double (nullable = true)
 |-- order_date: date (nullable = true)
 |-- ship_date: date (nullable = true)
 |-- payment_date: date (nullable = true)



In [11]:
spark.sql("DROP DATABASE IF EXISTS bronze_kayden CASCADE")
spark.sql("DROP DATABASE IF EXISTS silver_kayden CASCADE")
spark.sql("DROP DATABASE IF EXISTS gold_kayden CASCADE")

DataFrame[]

In [12]:
kayden_df = spark.sql("SELECT * FROM gold_kayden.shopee_fact_sales")

kayden_df.printSchema()

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `gold_kayden`.`shopee_fact_sales` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.; line 1 pos 14;
'Project [*]
+- 'UnresolvedRelation [gold_kayden, shopee_fact_sales], [], false


In [13]:
spark.sql("SHOW TABLES IN gold_kayden")

DataFrame[namespace: string, tableName: string, isTemporary: boolean]